Implementation of EAST and CRAFT using Python. This code is part of the paper "Text Detection in Natural Scene Images using CRAFT and EAST" by Henry Febrian for the module Computer Graphics and Image Processing by Prof. Erik Rodner.

1. Imports. The corresponding files are already available in the 'bvlecture' environment, excluding imutils. It can be installed using 'pip install imutils'.

In [1]:
import os, sys
import numpy as np
import cv2
import time
from imutils.object_detection import non_max_suppression

Function for EAST detection

In [ ]:
def east(img):
    image = img.copy # Storing the original image to be used later
    
    # The image will need to be 'prepped' before being passed into EAST